# Greenplum Database  Concepts Explained (Part 4)

This is Part 4 of Greenplum Database  Concepts Explained, ***Table Storage Models***. 

- If you missed Part 1 (*Setup, Describe Input Dataset & Data Loading*) or wish to repeat, then click [here](AWS-GP-demo-1.ipynb).
- If you missed Part 2 (*Basic Table Functions*) or wish to repeat, then click [here](AWS-GP-demo-2.ipynb).
- If you missed Part 3 (*MPP Fundamentals and Partitioning*) or wish to repeat, then click [here](AWS-GP-demo-3.ipynb).

In [48]:
import os, re
from IPython.display import display_html

import pygments.lexers
from pygments import highlight
from pygments.formatters import HtmlFormatter

CONNECTION_STRING = os.getenv('AWSGPDBCONN')

cs = re.match('^postgresql:\/\/(\S+):(\S+)@(\S+):(\S+)\/(\S+)$', CONNECTION_STRING)

DB_USER   = cs.group(1)
DB_PWD    = cs.group(2)
DB_SERVER = cs.group(3)
DB_PORT   = cs.group(4)
DB_NAME   = cs.group(5)

%reload_ext sql
%sql $CONNECTION_STRING

In [49]:
%%sql $DB_USER@$DB_NAME
SHOW gp_autostats_mode;
ALTER DATABASE dev SET gp_autostats_mode TO 'NONE';
SHOW gp_autostats_mode;

1 rows affected.
Done.
1 rows affected.


gp_autostats_mode
none


In [50]:
query = !cat script/7-db-maintenance.sql
%sql $DB_USER@$DB_NAME {''.join(query)}

Done.
Done.


[]

## 7. Table Storage Models

### 7.1. Comparing Greenplum Table Storage Models: Loading

Re-create the Amazon Reviews table, using 3 different table storage models, Heap table, Append-Optimized (AO)/Row-Oriented table with ZLib (Level 3) compression, and Append-Optimized (AO)/Column-Oriented table with ZLib (Level 3) compression, as shown below:

In [60]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-1-amzn-reviews-ro.sql
sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-1-amzn-reviews-co.sql

display_html('\n'.join(sqlfilecode1), raw=True)
display_html('\n'.join(sqlfilecode3), raw=True)

query1 = !cat script/7-1-amzn-reviews-ro.sql
query3 = !cat script/7-1-amzn-reviews-co.sql

%sql $DB_USER@$DB_SERVER {''.join(query1)}
%sql $DB_USER@$DB_SERVER {''.join(query3)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP TABLE IF EXISTS demo . amzn_reviews_ro ; 

 CREATE TABLE demo . amzn_reviews_ro ( 
 marketplace TEXT , 
 customer_id BIGINT , 
 review_id TEXT , 
 product_id TEXT , 
 product_parent BIGINT , 
 product_title TEXT , 
 product_category TEXT , 
 star_rating INTEGER , 
 helpful_votes INTEGER , 
 total_votes INTEGER , 
 vine TEXT , 
 verified_purchase TEXT , 
 review_headline TEXT , 
 review_body TEXT , 
 review_date DATE ) 
 DISTRIBUTED BY ( review_id );

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP TABLE IF EXISTS demo . amzn_reviews_co ; 

 CREATE TABLE demo . amzn_reviews_co ( 
 marketplace TEXT , 
 customer_id BIGINT , 
 review_id TEXT , 
 product_id TEXT , 
 product_parent BIGINT , 
 product_title TEXT , 
 product_category TEXT , 
 star_rating INTEGER , 
 helpful_votes INTEGER , 
 total_votes INTEGER , 
 vine TEXT , 
 verified_purchase TEXT , 
 review_headline TEXT , 
 review_body TEXT , 
 review_date DATE ) 
 WITH ( appendonly = true , orientation = column , compresstype = zlib , compresslevel = 3 ) 
 DISTRIBUTED BY ( review_id );

Done.
Done.
Done.
Done.


[]

Load the input dataset to each using the `gpload` utility, and compare loading times.

In [61]:
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'if [ -f ./gpload-amzn-reviews-ro.log ]; then rm ./gpload-amzn-reviews-ro.log; fi'
!scp -i ~/.ssh/aws-gp.pem script/7-1-gpload-amzn-reviews-ro.yaml $DB_USER@$DB_SERVER:gpload-amzn-reviews-ro.yaml
cmd = "gpload -d {0} -f ./gpload-amzn-reviews-ro.yaml -l ./gpload-amzn-reviews-ro.log 2>&1".format(DB_NAME) 
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd

7-1-gpload-amzn-reviews-ro.yaml               100%  375    41.3KB/s   00:00    
2020-06-19 17:44:19|INFO|gpload session started 2020-06-19 17:44:19
2020-06-19 17:44:19|INFO|no host supplied, defaulting to localhost
2020-06-19 17:44:19|INFO|started gpfdist -p 8000 -P 9000 -f "/var/tmp_s3_data/amazon_reviews_us*.tsv.gz" -t 30 -m 1000000
2020-06-19 17:44:19|INFO|did not find an external table to reuse. creating ext_gpload_reusable_1f0a1ec8_b24c_11ea_8191_06481cc79c18
2020-06-19 17:50:40|WARN|4835 bad rows
2020-06-19 17:50:40|WARN|Please use following query to access the detailed error
2020-06-19 17:50:40|WARN|select * from gp_read_error_log('ext_gpload_reusable_1f0a1ec8_b24c_11ea_8191_06481cc79c18') where cmdtime > to_timestamp('1592585059.62')
2020-06-19 17:50:40|INFO|running time: 380.42 seconds
2020-06-19 17:50:40|INFO|rows Inserted          = 97238506
2020-06-19 17:50:40|INFO|rows Updated           = 0
2020-06-19 17:50:40|INFO|data formatting errors = 4835
2020-06-19 17:50:40|INFO|gpl

In [62]:
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER 'if [ -f ./gpload-amzn-reviews-co.log ]; then rm ./gpload-amzn-reviews-co.log; fi'
!scp -i ~/.ssh/aws-gp.pem script/7-1-gpload-amzn-reviews-co.yaml $DB_USER@$DB_SERVER:gpload-amzn-reviews-co.yaml
cmd = "gpload -d {0} -f ./gpload-amzn-reviews-co.yaml -l ./gpload-amzn-reviews-co.log 2>&1".format(DB_NAME) 
!ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd

7-1-gpload-amzn-reviews-co.yaml               100%  374    42.1KB/s   00:00    
2020-06-19 17:50:41|INFO|gpload session started 2020-06-19 17:50:41
2020-06-19 17:50:41|INFO|no host supplied, defaulting to localhost
2020-06-19 17:50:41|INFO|started gpfdist -p 8000 -P 9000 -f "/var/tmp_s3_data/amazon_reviews_us*.tsv.gz" -t 30 -m 1000000
2020-06-19 17:50:41|INFO|did not find an external table to reuse. creating ext_gpload_reusable_02afd87a_b24d_11ea_8aa6_06481cc79c18
2020-06-19 17:57:01|WARN|4835 bad rows
2020-06-19 17:57:01|WARN|Please use following query to access the detailed error
2020-06-19 17:57:01|WARN|select * from gp_read_error_log('ext_gpload_reusable_02afd87a_b24d_11ea_8aa6_06481cc79c18') where cmdtime > to_timestamp('1592585441.55')
2020-06-19 17:57:01|INFO|running time: 380.35 seconds
2020-06-19 17:57:01|INFO|rows Inserted          = 97238506
2020-06-19 17:57:01|INFO|rows Updated           = 0
2020-06-19 17:57:01|INFO|data formatting errors = 4835
2020-06-19 17:57:01|INFO|gpl

In [64]:
cmd = 'grep -e '"'"'running'"'"' /home/gpadmin/gpload-amzn-reviews*\
    | awk '"'"'BEGIN{FS=":"} {print $1, "finished in", $5}'"'"'' 
grep_output = !ssh -i ~/.ssh/aws-gp.pem $DB_USER@$DB_SERVER $cmd | pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(grep_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 / home / gpadmin / gpload - amzn - reviews - co . log finished in 380.35 seconds 
 / home / gpadmin / gpload - amzn - reviews - ro . log finished in 380.42 seconds

### 7.2. Comparing Greenplum Table Storage Models: Table Size and Disk Space Usage

In [65]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/7-2-table-size-comparison.sql
display_html('\n'.join(sqlfilecode), raw=True)
query = !cat script/7-2-table-size-comparison.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT pg_namespace . nspname AS schema , 
 pg_class . relname AS relation , 
 pg_size_pretty ( sotd . sotdsize :: BIGINT ) as tablesize , 
 pg_size_pretty ( sotd . sotdtoastsize :: BIGINT ) as toastsize , 
 pg_size_pretty ( sotd . sotdadditionalsize :: BIGINT ) as othersize , 
 pg_size_pretty ( sotaid . sotaidtablesize :: BIGINT ) as tabledisksize , 
 pg_size_pretty ( sotaid . sotaididxsize :: BIGINT ) as indexsize , 
 pg_size_pretty ( sotu . sotusize :: BIGINT ) as uncompressedsize , 
 ROUND (( 100.0 * ( 1 - ( sotaid . sotaidtablesize / sotu . sotusize ))) :: NUMERIC , 2 ) AS compressionpercentage 
 FROM pg_class 
 LEFT JOIN pg_stat_user_tables 
 ON pg_stat_user_tables . relid = pg_class . oid 
 LEFT JOIN gp_toolkit . gp_size_of_table_disk sotd 
 ON sotd . sotdoid = pg_class . oid 
 LEFT JOIN gp_toolkit . gp_size_of_table_and_indexes_disk sotaid 
 ON sotaid . sotaidoid = pg_class . oid 
 LEFT JOIN gp_toolkit . gp_size_of_table_uncompressed sotu 
 ON sotu . sotuoid = pg_class . oid 
 LEFT JOIN pg_namespace 
 ON pg_namespace . oid = pg_class . relnamespace 
 WHERE 
 pg_class . relkind = 'r' 
 AND relstorage != 'x' 
 AND pg_namespace . nspname NOT IN ( 'information_schema' , 'madlib' , 'pg_catalog' , 'gptext' ) 
 AND pg_class . relname NOT IN ( 'spatial_ref_sys' ) 
 ORDER BY 1 , 2

2 rows affected.


schema,relation,tablesize,toastsize,othersize,tabledisksize,indexsize,uncompressedsize,compressionpercentage
demo,amzn_reviews_co,23 GB,800 kB,1600 kB,23 GB,0 bytes,53 GB,56.14
demo,amzn_reviews_ro,56 GB,193 MB,0 bytes,57 GB,0 bytes,57 GB,0.00


### 7.3. Comparing Greenplum Table Storage Models: Query Performance

#### 7.3.0. `ANALYZE` tables

In [78]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-analyze.sql
display_html('\n'.join(sqlfilecode), raw=True)
query = !cat script/7-3-analyze.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 ANALYZE demo . amzn_reviews_ro ; 
 ANALYZE demo . amzn_reviews_co ;

Done.
Done.


[]

#### 7.3.1. Narrow (*Few columns of the table*) `SELECT`

In [79]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-narrow-select-ro.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-narrow-select-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-narrow-select-co.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-narrow-select-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT product_id , 
 marketplace , 
 product_category , 
 star_rating 
 FROM demo . amzn_reviews_ro 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY RANDOM ();

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Execution time : 5806.176 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT product_id , 
 marketplace , 
 product_category , 
 star_rating 
 FROM demo . amzn_reviews_co 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY RANDOM ();

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Execution time : 4973.231 ms

#### 7.3.2. Super Narrow (*1 column of the table*) `SELECT`

In [80]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-super-narrow-select-ro.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-super-narrow-select-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-super-narrow-select-co.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-super-narrow-select-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT product_id 
 FROM demo . amzn_reviews_ro 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY RANDOM ();

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Execution time : 5060.718 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT product_id 
 FROM demo . amzn_reviews_co 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY RANDOM ();

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Execution time : 4472.933 ms

#### 7.3.3. Wide (*Most/Many columns of the table*) `SELECT`

In [81]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-wide-select-ro.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-wide-select-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-wide-select-co.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-wide-select-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT marketplace , 
 customer_id , 
 product_id , 
 product_title , 
 product_parent , 
 product_category , 
 star_rating , 
 helpful_votes , 
 total_votes , 
 vine , 
 verified_purchase , 
 review_headline , 
 review_body 
 FROM 
 demo . amzn_reviews_ro 
 WHERE 
 DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY 
 RANDOM () 
 LIMIT 2000 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Execution time : 2179.238 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT marketplace , 
 customer_id , 
 product_id , 
 product_title , 
 product_parent , 
 product_category , 
 star_rating , 
 helpful_votes , 
 total_votes , 
 vine , 
 verified_purchase , 
 review_headline , 
 review_body 
 FROM 
 demo . amzn_reviews_co 
 WHERE 
 DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 ORDER BY 
 RANDOM () 
 LIMIT 2000 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Execution time : 14107.222 ms

#### 7.3.4. Aggregate/Window Functions over a limited number of columns

In [82]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-aggr-select-ro.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-aggr-select-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-aggr-select-co.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-aggr-select-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ), 
 product_category , 
 star_rating 
 FROM demo . amzn_reviews_ro 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 GROUP BY 2 , 3 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Execution time : 3905.316 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ), 
 product_category , 
 star_rating 
 FROM demo . amzn_reviews_co 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 GROUP BY 2 , 3 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Execution time : 2382.766 ms

#### 7.3.5 Aggregate/Window Functions over a more columns

In [83]:
sqlfilecode1 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-aggr-select-ro-2.sql
display_html('\n'.join(sqlfilecode1), raw=True)
cmd1 = !echo $(cat script/7-3-aggr-select-ro.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd1), raw=True)

sqlfilecode3 = !pygmentize -f html -O full,style=colorful -l postgres script/7-3-aggr-select-co-2.sql
display_html('\n'.join(sqlfilecode3), raw=True)
cmd3 = !echo $(cat script/7-3-aggr-select-co.sql | \
               psql $CONNECTION_STRING | \
               grep -e 'Execution time') | \
    pygmentize -f html -O full,style=colorful -l postgres
display_html('\n'.join(cmd3), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ), 
 ROUND ( AVG ( helpful_votes )) AS helpful_votes_avg , 
 ROUND ( AVG ( total_votes )) AS total_votes_Avg , 
 marketplace , 
 product_category , 
 star_rating , 
 verified_purchase , 
 CASE WHEN LENGTH ( review_headline ) <= 20 THEN 'Short Headline' ELSE 'Long Headline' END AS review_headline_length_type , 
 CASE WHEN LENGTH ( review_body ) <= 400 THEN 'Short Body' ELSE 'Long Body' END AS review_body_length_type 
 FROM demo . amzn_reviews_ro 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 GROUP BY 4 , 5 , 6 , 7 , 8 , 9 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Execution time : 3511.547 ms

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ), 
 ROUND ( AVG ( helpful_votes )) AS helpful_votes_avg , 
 ROUND ( AVG ( total_votes )) AS total_votes_Avg , 
 marketplace , 
 product_category , 
 star_rating , 
 verified_purchase , 
 CASE WHEN LENGTH ( review_headline ) <= 20 THEN 'Short Headline' ELSE 'Long Headline' END AS review_headline_length_type , 
 CASE WHEN LENGTH ( review_body ) <= 400 THEN 'Short Body' ELSE 'Long Body' END AS review_body_length_type 
 FROM demo . amzn_reviews_co 
 WHERE DATE_PART ( 'year' , review_date ) BETWEEN 2000 AND 2005 
 GROUP BY 4 , 5 , 6 , 7 , 8 , 9 ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 Execution time : 1746.500 ms